# Intersection with a crosswalk

**Load dependencies**

In [1]:
using Revise
using Random
using Printf
using Flux
using POMDPs
using POMDPModelTools
using POMDPSimulators
using BeliefUpdaters
using POMDPPolicies
using DiscreteValueIteration
using MDPModelChecking
using StaticArrays
using DeepRL
using TensorFlow
using DeepQLearning
using AutomotiveDrivingModels
using AutomotivePOMDPs
using AutomotiveSensors
using LocalApproximationValueIteration
using Reel
using AutoViz
using ProgressMeter
using JLD2
using FileIO
using BSON
using PedCar

loaded


┌ Info: Recompiling stale cache file /mnt/c/Users/Maxime/wsl/.julia/compiled/v1.0/PedCar/NmDDZ.ji for PedCar [90cf7f26-d5c7-593d-a0e1-4a8367407571]
└ @ Base loading.jl:1184
┌ Warning: Package PedCar does not have AutomotivePOMDPs in its dependencies:
│ - If you have PedCar checked out for development and have
│   added AutomotivePOMDPs as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PedCar
└ Loading AutomotivePOMDPs into PedCar from project dependency, future warnings for PedCar are suppressed.


In [2]:
include("masking.jl")
include("util.jl")
include("masked_dqn.jl")
include("qmdp_approximation.jl")
include("render_helpers.jl")

In [3]:
rng = MersenneTwister(1);
cam = FitToContentCamera(0.);

## Scenario

In [4]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

## Discrete MDP

In [5]:
mdp = PedCarMDP(env=env, pos_res=2.0, vel_res=2., ped_birth=0.7, car_birth=0.7);
init_transition!(mdp);

In [6]:
@printf("spatial resolution %2.1f m \n", mdp.pos_res)
@printf("pedestrian velocity resolution %2.1f m/s \n", mdp.vel_ped_res)
@printf("car velocity resolution %2.1f m/s \n", mdp.vel_res)
@printf("number of states %d \n", n_states(mdp))
@printf("number of actions %d \n", n_actions(mdp))

spatial resolution 2.0 m 
pedestrian velocity resolution 1.0 m/s 
car velocity resolution 2.0 m/s 
number of states 23456940 
number of actions 4 


In [7]:
@load "pc_util_processed.jld2" qmat util pol
safe_policy = ValueIterationPolicy(mdp, qmat, util, pol);

In [8]:
threshold = 0.99
mask = SafetyMask(mdp, safe_policy, threshold);
rand_pol = MaskedEpsGreedyPolicy(mdp, 1.0, mask, rng);

In [8]:
# Load VI data for maksing
@time state_space = states(mdp);
vi_data = load("pedcar_utility.jld2")
@showprogress for s in state_space
    if !s.crash && isterminal(mdp, s)
        si = stateindex(mdp, s)
        vi_data["util"][si] = 1.0
        vi_data["qmat"][si, :] = ones(n_actions(mdp))
    end
end
policy = ValueIterationPolicy(mdp, vi_data["qmat"], vi_data["util"], vi_data["pol"]);
util = policy.util
qmat = policy.qmat
pol = policy.policy 
@save "pc_util_processed.jld2" util qmat pol

Progress: 100%|█████████████████████████████████████████| Time: 0:00:3839m


In [9]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initialstate(mdp, rng)
@time hist2 = simulate(hr, mdp, safe_policy, s0);

  1.916927 seconds (3.43 M allocations: 170.618 MiB, 9.74% gc time)


In [10]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
push!(action_hist, UrbanAction(NaN))
duration, fps, render_hist = animate_states(mdp, state_hist, action_hist, mask)
film = roll(render_hist, fps = fps, duration = duration)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpHQYYnE", 0x0000000000000018, 2.0, nothing)

In [26]:
@showprogress for ep=1:10000
    global hist2
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(mdp, rng)
    hist2 = simulate(hr, mdp, rand_pol, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Progress:   0%|                                         |  ETA: 0:20:39

Crash


Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


**Evaluation**

In [22]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(mdp, safe_policy, n_ep=10000, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress:  99%|█████████████████████████████████████████|  ETA: 0:00:00

 28.871124 seconds (50.59 M allocations: 8.884 GiB, 6.48% gc time)
Summary for 10000 episodes: 
Average reward: 0.191 
Average # of steps: 43.404 
Average # of violations: 0.000 


Progress: 100%|█████████████████████████████████████████| Time: 0:00:29


## Continuous state MDP

In [15]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [16]:
# sensor
pomdp = UrbanPOMDP(env=env,
                   sensor = PerfectSensor(),
                   ego_goal = LaneTag(2, 1),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.7, 
                   max_obstacles=0., # no fixed obstacles
                   lidar=false,
                   ego_start=20,
                   ΔT=0.1);

In [148]:
function POMDPModelTools.generate_sori(pomdp::UrbanPOMDP, s::Scene, a::UrbanAction, rng::AbstractRNG)
    sp, o, r = generate_sor(pomdp, s, a, rng)
    return sp, o, r, deepcopy(pomdp.models)
end

In [149]:
rand_pol = RandomMaskedPOMDPPolicy(mask, pomdp, rng);
# rand_pol = solve(RandomSolver(), mdp);

In [150]:
hr = HistoryRecorder(rng=rng, max_steps=400)
s0 = initialstate(pomdp,rng)
o0 = generate_o(pomdp, s0, rng)
up = PreviousObservationUpdater()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0);

  0.362397 seconds (1.49 M allocations: 113.540 MiB, 11.81% gc time)


In [154]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
belief_hist = h.belief_hist
safe_acts = [i[1] for i in h.ainfo_hist]
probas = [i[2] for i in h.ainfo_hist]
routes = [i[3] for i in h.ainfo_hist]

push!(safe_acts, [UrbanAction(NaN)])
push!(probas, [NaN])
push!(routes, PedCar.OFF_ROUTE)
push!(action_hist, UrbanAction(NaN))
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, belief_hist, safe_acts, probas, routes, mask, interp=false, obsviz=true)
film = roll(render_hist, fps = fps, duration = duration)

Frames{MIME{Symbol("image/png")}}("/tmp/tmphMwBTt", 0x000000000000004e, 10.0, nothing)

In [141]:
step = 79
s = h.state_hist[step]
vehid = 2
veh = s[findfirst(vehid, s)]
m = h.info_hist[step][vehid]
cwid = 1
cwm = m.crosswalk_drivers[cwid]
intm = m.intersection_driver

TTCIntersectionDriver
  a: LonAccelDirection
  navigator: RouteFollowingIDM
  intersection: Array{Lane}((2,))
  intersection_pos: VecSE2{Float64}
  ttc_threshold: Float64 3.0
  horizon: Float64 20.0
  stop_delta: Float64 4.0
  accel_tol: Float64 0.0
  priorities: Dict{Tuple{LaneTag,LaneTag},Bool}
  priority: Bool false
  stop: Bool false


In [142]:
AutomotivePOMDPs.ttc_check(intm, s, env.roadway, vehid)

false

In [143]:
AutomotivePOMDPs.engaged(intm, s, env.roadway, vehid)

false

In [144]:
AutomotivePOMDPs.has_passed(intm, s, env.roadway, vehid)

false

In [145]:
intm.a

LonAccelDirection(0.03125, 2)

In [126]:
ego = s[findfirst(EGO_ID, s)]
sqrt(normsquared(VecE2(veh.state.posG - ego.state.posG)))

3.7319421333163305

In [127]:
veh.state.v

0.0

In [113]:
mm = deepcopy(intm)
observe!(mm, s, env.roadway, vehid)
mm.a

LonAccelDirection(0.125, 2)

In [104]:
get_lane(env.roadway, veh)
veh.state

VehicleState(VecSE2({-6.453, -1.500}, 0.000), Frenet(RoadIndex({1, 0.981120}, {3, 1}), 23.547, 0.000, 0.000), 7.938)

In [46]:
get_end(get_lane(env.roadway, veh))

24.0

In [100]:
function AutomotivePOMDPs.ttc_check(model::TTCIntersectionDriver, scene::Scene, roadway::Roadway, egoid::Int)
    min_ttc = Inf
    inter_width = 6.0 #todo parameterized
    otherid = -1
    ego = scene[findfirst(egoid, scene)]
    for veh in scene
        if veh.id != egoid && veh.def.class != AgentClass.PEDESTRIAN && !AutomotivePOMDPs.is_behind(ego, veh, roadway)
            posF = veh.state.posF
            int_x, int_y, int_θ = model.intersection_pos
            lane = get_lane(roadway, veh)
            int_proj = Frenet(model.intersection_pos, lane, roadway)
            if normsquared(VecE2(model.intersection_pos - veh.state.posG)) < inter_width^2 # vehicle is in the middle
                println(normsquared(VecE2(model.intersection_pos - veh.state.posG)))
                ttc = 0.
            else
                ttc = (int_proj.s - posF.s)/veh.state.v
            end
            if 0 <= ttc < min_ttc
                otherid = veh.id
                min_ttc = ttc
            end
        end
    end
    println("veh id ", egoid, "min_ttc ", min_ttc, " threshold ", model.ttc_threshold, " other id ", otherid)
    if 0 <= min_ttc < model.ttc_threshold
        return false
    else
        return true
    end
end

In [22]:
include("masking.jl")
include("render_helpers.jl")

In [66]:
fieldnames(LaneTag)

2-element Array{Symbol,1}:
 :segment
 :lane   

In [99]:
s = h.state_hist[2]
b = h.belief_hist[2]
ai = h.ainfo_hist[2]
a = h.action_hist[2]
# println(ai[1])
# println(ai[2])
# println(a)
# println(safe_actions(pomdp, rand_pol.mask, s, 101, 2))
# println(safe_actions(pomdp, rand_pol.mask, b))
action(rand_pol, b)
# for veh in s
#     println(veh)
# end
s_mdp = get_mdp_state(rand_pol.mask.mdp, rand_pol.pomdp, s, PED_ID, CAR_ID)
itp_ped, itp_ped_w = interpolate_pedestrian(mdp, s_mdp.ped)
for ped in itp_ped
    println(ped)
end
println(s_mdp.ped)

VehicleState(VecSE2({-7.000, -5.000}, 0.000), Frenet(RoadIndex({1, 0.000000}, {19, 1}), 0.000, 0.000, 0.000), 1.000)
VehicleState(VecSE2({-5.000, -5.000}, 0.000), Frenet(RoadIndex({1, 0.142857}, {19, 1}), 2.000, 0.000, 0.000), 1.000)
VehicleState(VecSE2({-6.079, -5.423}, 0.000), Frenet(RoadIndex({1, 0.065755}, {19, 1}), 0.921, -0.423, 0.000), 1.000)


In [78]:
s_mdp.ped

VehicleState(VecSE2({-28.000, -29.100}, 0.000), Frenet(RoadIndex({1, 0.000000}, {18, 1}), 0.000, 22.000, -1.571), 0.000)

In [68]:
model = h.info_hist[57][2]
model.a


LoadError: [91mBoundsError: attempt to access 44-element Array{Any,1} at index [57][39m

In [64]:
[m.a for m in model.crosswalk_drivers]
m = model.intersection_driver
m.priority

true

In [61]:
AutomotivePOMDPs.engaged(m, state_hist[57], env.roadway, 2)

true

In [63]:
# AutomotivePOMDPs.engaged(model, state_hist[25], env.roadway, 2)
observe!(model, state_hist[25], env.roadway, 2)

AutomotivePOMDPs.UrbanDriver
  a: AutomotivePOMDPs.LonAccelDirection
  navigator: AutomotivePOMDPs.RouteFollowingIDM
  intersection_driver: AutomotivePOMDPs.TTCIntersectionDriver
  crosswalk_drivers: Array{AutomotivePOMDPs.CrosswalkDriver}((3,))
  debug: Bool false


In [198]:
sc = state_hist[1]
b = h.belief_hist[1]
safe_actions(pomdp, mask, sc, 101, 2)

4-element Array{AutomotivePOMDPs.UrbanAction,1}:
 AutomotivePOMDPs.UrbanAction(-4.0)
 AutomotivePOMDPs.UrbanAction(-2.0)
 AutomotivePOMDPs.UrbanAction(0.0) 
 AutomotivePOMDPs.UrbanAction(2.0) 

In [152]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, rand_pol, n_ep=1000, max_steps=400, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:01

893.832134 seconds (5.01 G allocations: 380.725 GiB, 18.74% gc time)
Summary for 1000 episodes: 
Average reward: -0.000 
Average # of steps: 388.069 
Average # of violations: 1.500 


Progress: 100%|█████████████████████████████████████████| Time: 0:14:54


In [153]:
@showprogress for ep=1:10000
    global hist2
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(pomdp, rng)
    o0 = generate_o(pomdp, s0, rng)
    up = PreviousObservationUpdater()
    b0 = initialize_belief(up, o0)
    hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Progress:  21%|█████████                                |  ETA: 0:28:55

Crash


Progress: 100%|█████████████████████████████████████████| Time: 0:07:54


## Trained Policy

In [17]:
include("masked_dqn.jl")

In [18]:
problem_file="training_scripts/drqn-log/log8/problem.bson"
weights_file="training_scripts/drqn-log/log8/weights.bson"
env_ = POMDPEnvironment(pomdp)
dqn_policy = DeepQLearning.restore(env_, problem_file=problem_file, weights_file=weights_file)
policy = MaskedNNPolicy(pomdp, dqn_policy, mask);

2018-10-17 21:48:20.729020: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


In [23]:
hr = HistoryRecorder(rng=rng, max_steps=400)
s0 = initialstate(pomdp,rng)
o0 = generate_o(pomdp, s0, rng)
up = PreviousObservationUpdater()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, policy, up, b0, s0);

  0.171773 seconds (907.58 k allocations: 66.687 MiB, 14.02% gc time)


In [24]:
include("render_helpers.jl")

In [25]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
belief_hist = h.belief_hist
safe_acts = [i[1] for i in h.ainfo_hist]
probas = [i[2] for i in h.ainfo_hist]
routes = [i[3] for i in h.ainfo_hist]

push!(safe_acts, [UrbanAction(NaN)])
push!(probas, [NaN])
push!(routes, PedCar.OFF_ROUTE)
push!(action_hist, UrbanAction(NaN))
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, belief_hist, safe_acts, probas, routes, mask, interp=true, obsviz=true)
film = roll(render_hist, fps = fps, duration = duration)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpcmbSZ5", 0x000000000000004a, 10.0, nothing)

In [85]:
function MDPModelChecking.safe_actions(pomdp::UrbanPOMDP, mask::SafetyMask{PedCarMDP, P}, s::UrbanState, ped_id, car_id) where P <: Policy
    s_mdp = get_mdp_state(mask.mdp, pomdp, s, ped_id, car_id)
    itp_states, itp_weights = interpolate_state(mask.mdp, s_mdp)
    action_space = actions(mask.mdp)
    # compute risk vector
    p_sa = zeros(n_actions(mask.mdp))
    for (i, ss) in enumerate(itp_states)
        vals = value_vector(mask.policy, ss)
        p_sa += itp_weights[i]*vals
    end
#     println(p_sa)
    safe_acts = UrbanAction[]
    sizehint!(safe_acts, n_actions(mask.mdp))
    if maximum(p_sa) <= mask.threshold
        push!(safe_acts, action_space[argmax(p_sa)])
    else
        for (j, a) in enumerate(action_space)
            if p_sa[j] > mask.threshold
                push!(safe_acts, a)
            end
        end
    end
    return safe_acts
end

safe_actions(pomdp, mask, state_hist[1], 101, 2)

4-element Array{AutomotivePOMDPs.UrbanAction,1}:
 AutomotivePOMDPs.UrbanAction(-4.0)
 AutomotivePOMDPs.UrbanAction(-2.0)
 AutomotivePOMDPs.UrbanAction(0.0) 
 AutomotivePOMDPs.UrbanAction(2.0) 

In [22]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, policy, n_ep=1000, max_steps=400, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:00

267.883687 seconds (1.28 G allocations: 91.604 GiB, 13.64% gc time)
Summary for 1000 episodes: 
Average reward: 0.018 
Average # of steps: 106.310 
Average # of violations: 0.100 


Progress: 100%|█████████████████████████████████████████| Time: 0:04:28
